# Aprendizaje supervisado

In [1]:
# En estos notebooks vamos implementar la clasificación de digitos de la serie MNIST
# En la mayoría de nuestros ejemplos utilizaremos la librería de Machine Learning 
# Scikit-Learn.

In [1]:
# Importamos nuestra librerías básicas
import numpy as np
import os

# Este paso es para que los resultados en vuestros notebooks sean iguales a lo de este
np.random.seed(42)


In [2]:
# Aqui importamos los datos que vamos a clasificar

# Scikit-Learn ya incluye algunos datasets de ejemplo como el MNIST
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')

# El dataset contiene 70.000 ejemplos de digitos escritos a mano
# en blanco y negro. Cada digito esta representada por una imagen de 28x28 pixels.
# Además el dataset incluye el "target" i.e. el numero asociado con cada imagen.

# Otros datasets serían más rápidos de ejecutar, pero la ventaja de este es que,
# debido a su complejidad, cuando empecemos a utilizar modelos más complejos (como 
# redes neuronales), se empezarán a ver las ventajas de estos modelos más complejos

# Aqui cargamos nuestros ejemplos en X, el target en y. Nuestro objetivo con
# Machine Learning es aprender la función f(X) que genera y.
X, y = mnist["data"], mnist["target"]

In [3]:
# Nuestro objetivo es que la función aprendida funcione no solamente con
# el dataset de prueba, pero que también "generalize" bien para ejemplos
# que no haya visto antes.

# En esta sección separamos el dataset en 2 partes: el training set y test set
X_train_o, X_test_o, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

### Normalización de los datos (aka Feature Engineering)

In [4]:
# Antes de enviar los datos al modelo, hacemos una adaptación de los datos para normalizarlos
# Por normalización en este contexto la idea es remover la media de cada "feature"
# (i.e. centrar en zero) y dividir por la "variance", es decir hacer que los datos estén entre
# 0 y 1.

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_o)
X_test = scaler.transform(X_test_o)


C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


# Utilizando Scikit-Learn

## Regresión Logistica

In [7]:
# El problema que vamos a classificar tiene 768 dimensiones de entrada (28x28)
# La regresión logistica intentará definir una "línea" en este espacio multidimensional
# que mejor represente el dato

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV

# Creamos el modelo que queremos usar
# Para la regresión logistica, hemos indicado que el modelo es para una
# clasificacion de multiplas clases
lr_clf = LogisticRegression(multi_class='multinomial', solver='saga', tol=0.1)

# Hacemos fit del modelo al dato de training
%time lr_clf.fit(X_train, y_train)

# Los modelos de Scikit-Learn ya vienen con funciones para evaluar el rendimiento del modelo
# Con este modelo sencillo hemos podido alcanzar una "exactitud" de mis previsiones de casi 90%
# Usamos "exactitud" para diferenciar de precisión que significa otra cosa en nuestras medidas
# rendimiento.
%time train_score = lr_clf.score(X_train, y_train)
print("Rendimiento en el dataset de training: %.4f" % train_score)
%time score = lr_clf.score(X_test, y_test)
print("Rendimiento en el dataset de pruebas: %.4f" % score)


Wall time: 10.8 s
Wall time: 124 ms
Rendimiento en el dataset de training: 0.8892
Wall time: 19.1 ms
Rendimiento en el dataset de pruebas: 0.8917


## Clasificador utilizando Support Vector Machines

In [10]:
# El modelo SVC ...

from sklearn.svm import SVC

# Creamos el modelo
svc_clf = SVC(probability=True, random_state=42)

# Hacemos fit del modelo al dato
%time svc_clf.fit(X_train, y_train)

# Medimos el rendimiento.
%time train_score = svc_clf.score(X_train, y_train)
print("Rendimiento en el dataset de training: %.4f" % train_score)
%time score = svc_clf.score(X_test, y_test)
print("Rendimiento en el dataset de pruebas: %.4f" % score)

Wall time: 1h 5min 42s
Wall time: 16min 33s
Rendimiento en el dataset de training: 0.9853
Wall time: 2min 45s
Rendimiento en el dataset de pruebas: 0.9665


## Clasificador K-Neighbours

In [7]:
# En este metodo utilizamos una medida de proximidad del dato conocido ("training")
# para estimar las instancias de prueba

from sklearn.neighbors import KNeighborsClassifier

# Creamos el modelo. Los parametros del clasificador son, la cantidad de CPUs,
# numero de vecinos para utilizar y consider los puntos más cercanos como más
# importantes
knn_clf = KNeighborsClassifier(n_jobs=-1, weights='distance', n_neighbors=4)

# Hacemos fit del modelo al dato. Este metodo no requiere que las "features" estén
# normalizadas
%time knn_clf.fit(X_train_o, y_train)

# Medimos el rendimiento. Como este metodo, realmente procesa cada instancia de prueba
# contra todas las instancias conocidas, medir el rendimiento tarda más.
# Esto también significa que las predicciones de este metodo tardan más que el anterior.
%time train_score = knn_clf.score(X_train_o, y_train)
print("Rendimiento en el dataset de training: %.4f" % train_score)
% time score = knn_clf.score(X_test_o, y_test)
print("Rendimiento en el dataset de pruebas: %.4f" % score)



Wall time: 20.8 s
Wall time: 15min 34s
Rendimiento en el dataset de training: 1.0000
Wall time: 2min 34s
Rendimiento en el dataset de pruebas: 0.9714


## Clasificador utilizando Decision Tree

In [8]:
# En el modelo con las Decision Trees, el método consiste en crear un árbol ...

from sklearn.tree import DecisionTreeClassifier

# Creamos el modelo
tree_clf = DecisionTreeClassifier(random_state=42)

# Hacemos fit del modelo al dato
%time tree_clf.fit(X_train, y_train)

# Medimos el rendimiento. En este modelo la exactitud en el dataset de pruebas
# no es tan bueno cuando comparado con la exactitud alcanzada durante el entrenamiento.
# Esto se debe al "overfitting". Pero son modelos muy rápidos
%time train_score = tree_clf.score(X_train, y_train)
print("Rendimiento en el dataset de training: %.4f" % train_score)
%time score = tree_clf.score(X_test, y_test)
print("Rendimiento en el dataset de pruebas: %.4f" % score)

Wall time: 19.8 s
Wall time: 135 ms
Rendimiento en el dataset de training: 1.0000
Wall time: 25.1 ms
Rendimiento en el dataset de pruebas: 0.8756


## Clasificador utilizando Random Forests

In [9]:
# El modelo Random Forest, se construye sobre el de las Decision Trees
# En este modelo se construyen multiples árboles con parte de los datos
# de prueba, las previsiones son entonces medias de las previsiones de
# las diversas árboles, evitando el overfitting

from sklearn.ensemble import RandomForestClassifier

# Creamos el modelo
forest_clf = RandomForestClassifier(random_state=42)

# Hacemos fit del modelo al dato
%time forest_clf.fit(X_train, y_train)

# Medimos el rendimiento. En este modelo la exactitud en el dataset de pruebas
# no es tan bueno cuando comparado con la exactitud alcanzada durante el entrenamiento.
# Esto se debe al "overfitting".
%time train_score = forest_clf.score(X_train, y_train)
print("Rendimiento en el dataset de training: %.4f" % train_score)
%time score = forest_clf.score(X_test, y_test)
print("Rendimiento en el dataset de pruebas: %.4f" % score)

Wall time: 4.1 s
Wall time: 322 ms
Rendimiento en el dataset de training: 0.9991
Wall time: 55.1 ms
Rendimiento en el dataset de pruebas: 0.9455


# Utilizando Keras / Tensorflow


In [10]:
# En esta sección importamos el Tensorflow. Mas abajo importaremos el Keras.
# Estas librerías serán utilizadas para construir de forma sencilla
# las redes neuronales.

import tensorflow as tf


## Red neuronal sencilla

In [11]:
# En este ejemplo crearemos una red neuronal de uno sola capa
# "escondida" para ver cual exactitud podríamos llegar a obtener

from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD

# Creamos el modelo de red neuronal que queremos utilizar
def model_func():
    # Sequential indica que cada capa se ejecuta una detras de otra
    model = Sequential()
    # En las redes neuronales es particularmente importante inicializar correctamente los
    # parametros. Esto permite llegar a un modelo estable rápidamente
    # hemos seleccionado 1000 neuronas de forma arbitraria. Además utilizamos ReLU como
    # activacion.
    model.add(Dense(1000, kernel_initializer='he_normal', bias_initializer='he_normal', activation='relu', input_dim=784))
    # A continuación creamos la capa de salida que tiene las 10 categorías
    model.add(Dense(10, kernel_initializer='he_normal', bias_initializer='he_normal'))
    # Finalmente aplicamos una activación softmax que básicamente garantiza que la salida de esta capa
    # es una probabilidad distribuida en las 10 salidas de la red
    model.add(Activation('softmax'))
    # Aquí definimos cual es la función que queremos minimizar (esto es el error de clasificación
    # de cada categoría) y el algoritmo de aprendizaje (que en este caso es el Stochastic Gradient Descent)
    model.compile(loss='categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])
    return model

# Tensorflow incluye el Tensorboard que permite seguir el progreso del aprendizaje del modelo.
# Como estamos utilizando un modelo complejo, el Tensorboard nos permitirá ver si el modelo está
# progresando correctamente. En el caso que no, lo podríamos volver a arrancar.

from keras.callbacks import TensorBoard
from time import time

tensorboard=TensorBoard(log_dir="logs/{}".format(time()))

# El Keras nos proporciona un "wrapper" que permite que los clasificadores creados
# en Keras se comporten como clasificadores del Scikit-Learn. Esto permite seguir
# utilizando los mismos metodos que hemos usado hasta ahora para poder medir la
# exactitud del modelo

from keras.wrappers.scikit_learn import KerasClassifier

# Al definir el modelo decimos cuantas veces queremos mirar los datos (aquí hemos incluido
# 1000 veces) además de enviar 200 ejemplos de training de cada vez (en vez de 1 en 1 del
# SGD). Esto sería un Mini-batch SGD.    
nn_clf = KerasClassifier(build_fn=model_func, callbacks=[tensorboard], epochs=50, batch_size=1000, verbose=0)

# Hacemos fit del modelo al dato
%time nn_clf.fit(X_train, y_train)

# Medimos el rendimiento.
%time train_score = nn_clf.score(X_train, y_train)
print("Rendimiento en el dataset de training: %.4f" % train_score)
%time score = nn_clf.score(X_test, y_test)
print("Rendimiento en el dataset de pruebas: %.4f" % score)


Wall time: 40.3 s
Wall time: 532 ms
Rendimiento en el dataset de training: 0.9989
Wall time: 79.2 ms
Rendimiento en el dataset de pruebas: 0.9773


## Red Neuronal de multiplas capas

In [12]:
# En este ejemplo optimizaremos la red neuronal con multiplas capas y diversas
# funcionalidades avanzadas para garantizar el rendimiento

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.advanced_activations import ELU
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD

# Creamos el modelo de red neuronal que queremos utilizar
def model_func():
    # Sequential indica que cada capa se ejecuta una detras de otra
    model = Sequential()
    # En las redes neuronales es particularmente importante inicializar correctamente los
    # parametros. Esto permite llegar a un modelo estable rápidamente
    # hemos seleccionado 100 neuronas de forma arbitraria
    model.add(Dense(100, kernel_initializer='he_normal', bias_initializer='he_normal', activation='relu', input_dim=784))
    # Logo añadimos algunas capas de 100 neuronas para hacer una red "profunda" - deep learning
    model.add(Dense(100, kernel_initializer='he_normal', bias_initializer='he_normal', activation='relu'))
    model.add(Dense(100, kernel_initializer='he_normal', bias_initializer='he_normal', activation='relu'))
    model.add(Dense(100, kernel_initializer='he_normal', bias_initializer='he_normal', activation='relu'))
    # A continuación creamos la capa de salida que tiene las 10 categorías
    model.add(Dense(10, kernel_initializer='he_normal', bias_initializer='he_normal'))
    # Finalmente aplicamos una activación softmax que básicamente garantiza que la salida de esta capa
    # es una probabilidad distribuida en las 10 salidas de la red
    model.add(Activation('softmax'))
    # Aquí definimos cual es la función que queremos minimizar (esto es el error de clasificación
    # de cada categoría) y el algoritmo de aprendizaje (que en este caso es el Stochastic Gradient Descent)
    model.compile(loss='categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])
    return model

# Tensorflow incluye el Tensorboard que permite seguir el progreso del aprendizaje del modelo.
# Como estamos utilizando un modelo complejo, el Tensorboard nos permitirá ver si el modelo está
# progresando correctamente. En el caso que no, lo podríamos volver a arrancar.

from keras.callbacks import TensorBoard
from time import time

tensorboard=TensorBoard(log_dir="logs/{}".format(time()))

# El Keras nos proporciona un "wrapper" que permite que los clasificadores creados
# en Keras se comporten como clasificadores del Scikit-Learn. Esto permite seguir
# utilizando los mismos metodos que hemos usado hasta ahora para poder medir la
# exactitud del modelo

from keras.wrappers.scikit_learn import KerasClassifier

# Al definir el modelo decimos cuantas veces queremos mirar los datos (aquí hemos incluido
# 1000 veces) además de enviar 200 ejemplos de training de cada vez (en vez de 1 en 1 del
# SGD). Esto sería un Mini-batch SGD.    
nn_clf = KerasClassifier(build_fn=model_func, callbacks=[tensorboard], epochs=100, batch_size=1000, verbose=0)

# Hacemos fit del modelo al dato
%time nn_clf.fit(X_train, y_train)

# Medimos el rendimiento.
%time train_score = nn_clf.score(X_train, y_train)
print("Rendimiento en el dataset de training: %.4f" % train_score)
%time score = nn_clf.score(X_test, y_test)
print("Rendimiento en el dataset de pruebas: %.4f" % score)

Wall time: 1min 12s
Wall time: 505 ms
Rendimiento en el dataset de training: 0.9999
Wall time: 78.2 ms
Rendimiento en el dataset de pruebas: 0.9749


## Red Neuronal con una capa "convolutional"

In [14]:
# En este ejemplo optimizaremos la red neuronal con capas iniciales del tipo
# convolutional, para obtener la representacion y luego una capa
# "fully connected" para clasificar estas representaciones de alto nivel

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.advanced_activations import ELU
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD

# Creamos el modelo de red neuronal que queremos utilizar
def model_func():
    # Sequential indica que cada capa se ejecuta una detras de otra
    model = Sequential()
    # La capa convolutional ...
    model.add(Convolution2D(32, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=(28,28,1)))
    # La capa de pooling reduce la dimensionalidad de la imagen ...
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Convolution2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Finalmente la capa Flatten transforma el "shape" de las capas convolutional
    # en un vector para ser tratado por la capa "fully connected"
    model.add(Flatten())
    model.add(Dense(1000, kernel_initializer='he_normal', bias_initializer='he_normal', activation='relu'))
    # La capa de batch normalization ...
    model.add(BatchNormalization())
    # En este modelo hemos incluido Dropout como forma de "normalizar" las activaciones
    model.add(Dropout(0.2))
    # A continuación creamos la capa de salida que tiene las 10 categorías
    model.add(Dense(10, kernel_initializer='he_normal', bias_initializer='he_normal'))
    # Finalmente aplicamos una activación softmax que básicamente garantiza que la salida de esta capa
    # es una probabilidad distribuida en las 10 salidas de la red
    model.add(Activation('softmax'))
    # Aquí definimos cual es la función que queremos minimizar (esto es el error de clasificación
    # de cada categoría) y el algoritmo de aprendizaje (que en este caso es el Stochastic Gradient Descent)
    model.compile(loss='categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])
    return model

# La capa convolutional necesita un formato especifico de datos, para eso
# transformamos la entrada

X_train_shaped = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test_shaped = X_test.reshape(X_test.shape[0], 28, 28, 1)


# Tensorflow incluye el Tensorboard que permite seguir el progreso del aprendizaje del modelo.
# Como estamos utilizando un modelo complejo, el Tensorboard nos permitirá ver si el modelo está
# progresando correctamente. En el caso que no, lo podríamos volver a arrancar.

from keras.callbacks import TensorBoard
from time import time

tensorboard=TensorBoard(log_dir="logs/{}".format(time()))

# El Keras nos proporciona un "wrapper" que permite que los clasificadores creados
# en Keras se comporten como clasificadores del Scikit-Learn. Esto permite seguir
# utilizando los mismos metodos que hemos usado hasta ahora para poder medir la
# exactitud del modelo

from keras.wrappers.scikit_learn import KerasClassifier

# Al definir el modelo decimos cuantas veces queremos mirar los datos (aquí hemos incluido
# 1000 veces) además de enviar 200 ejemplos de training de cada vez (en vez de 1 en 1 del
# SGD). Esto sería un Mini-batch SGD.    
nn_clf = KerasClassifier(build_fn=model_func, callbacks=[tensorboard], epochs=50, batch_size=1000, verbose=0)

# Hacemos fit del modelo al dato
%time nn_clf.fit(X_train_shaped, y_train)

# Medimos el rendimiento.
%time train_score = nn_clf.score(X_train_shaped, y_train)
print("Rendimiento en el dataset de training: %.4f" % train_score)
%time score = nn_clf.score(X_test_shaped, y_test)
print("Rendimiento en el dataset de pruebas: %.4f" % score)

Wall time: 2min 16s
Wall time: 1.13 s
Rendimiento en el dataset de training: 0.9998
Wall time: 174 ms
Rendimiento en el dataset de pruebas: 0.9933
